In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from qiskit import Aer, QuantumCircuit
from qiskit.opflow import Z, I, StateFn
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B

from qiskit_machine_learning.neural_networks import TwoLayerQNN, CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR

from IPython.display import clear_output

from typing import Union

from qiskit_machine_learning.exceptions import QiskitMachineLearningError

algorithm_globals.random_seed = 42

In [2]:
#qircuit 01
circuit = QuantumCircuit(3)

#Input layer
circuit.ry(Parameter('i_1'),0)
circuit.ry(Parameter('i_2'),1)
circuit.ry(Parameter('i_3'),2)

#mixing/training layer, can very different. comparable to classical setup of NN, i.e. LSTM, hidden layers, activation functions. Still unclear which is good and what works best
circuit.rx(Parameter('w_1'),0)
circuit.crz(Parameter('w_12'),0,1)
circuit.rx(Parameter('w_2'),1)
circuit.crz(Parameter('w_23'),1,2)
circuit.rx(Parameter('w_3'),2)
circuit.crz(Parameter('w_13'),2,0)

circuit.draw()

┌─────────┐┌─────────┐                                              »
q_0: ┤ Ry(i_1) ├┤ Rx(w_1) ├─────■────────────────────────────────────────»
     ├─────────┤└─────────┘┌────┴─────┐┌─────────┐                       »
q_1: ┤ Ry(i_2) ├───────────┤ Rz(w_12) ├┤ Rx(w_2) ├─────■─────────────────»
     ├─────────┤           └──────────┘└─────────┘┌────┴─────┐┌─────────┐»
q_2: ┤ Ry(i_3) ├──────────────────────────────────┤ Rz(w_23) ├┤ Rx(w_3) ├»
     └─────────┘                                  └──────────┘└─────────┘»
«     ┌──────────┐
«q_0: ┤ Rz(w_13) ├
«     └────┬─────┘
«q_1: ─────┼──────
«          │      
«q_2: ─────■──────
«

In [3]:
#qircuit 02
circuit = QuantumCircuit(4)

#Input layer
circuit.ry(Parameter('i_1'),0)
circuit.ry(Parameter('i_2'),1)
circuit.ry(Parameter('i_3'),2)
circuit.ry(Parameter('i_4'),3)

#mixing/training layer, can very different. comparable to classical setup of NN, i.e. LSTM, hidden layers, activation functions. Still unclear which is good and what works best
circuit.rx(Parameter('w_1'),0)
circuit.rx(Parameter('w_2'),1)
circuit.rx(Parameter('w_3'),2)
circuit.rx(Parameter('w_4'),3)
circuit.crz(Parameter('w_12'),0,1)
circuit.crz(Parameter('w_23'),1,2)
circuit.crz(Parameter('w_34'),2,3)
circuit.crz(Parameter('w_13'),3,0)

circuit.draw()

┌─────────┐┌─────────┐                                    ┌──────────┐
q_0: ┤ Ry(i_1) ├┤ Rx(w_1) ├─────■──────────────────────────────┤ Rz(w_13) ├
     ├─────────┤├─────────┤┌────┴─────┐                        └────┬─────┘
q_1: ┤ Ry(i_2) ├┤ Rx(w_2) ├┤ Rz(w_12) ├─────■───────────────────────┼──────
     ├─────────┤├─────────┤└──────────┘┌────┴─────┐                 │      
q_2: ┤ Ry(i_3) ├┤ Rx(w_3) ├────────────┤ Rz(w_23) ├─────■───────────┼──────
     ├─────────┤├─────────┤            └──────────┘┌────┴─────┐     │      
q_3: ┤ Ry(i_4) ├┤ Rx(w_4) ├────────────────────────┤ Rz(w_34) ├─────■──────
     └─────────┘└─────────┘                        └──────────┘

In [4]:
#qircuit 04
circuit = QuantumCircuit(4)

#Input layer
circuit.ry(Parameter('i_1'),0)
circuit.ry(Parameter('i_2'),1)
circuit.ry(Parameter('i_3'),2)
circuit.ry(Parameter('i_4'),3)

#mixing/training layer, can very different. comparable to classical setup of NN, i.e. LSTM, hidden layers, activation functions. Still unclear which is good and what works best
circuit.rx(Parameter('w_x1'),0)
circuit.rx(Parameter('w_x2'),1)
circuit.rx(Parameter('w_x3'),2)
circuit.rx(Parameter('w_x4'),3)

circuit.ry(Parameter('w_y1'),0)
circuit.ry(Parameter('w_y2'),1)
circuit.ry(Parameter('w_y3'),2)
circuit.ry(Parameter('w_y4'),3)

circuit.rz(Parameter('w_z1'),0)
circuit.rz(Parameter('w_z2'),1)
circuit.rz(Parameter('w_z3'),2)
circuit.rz(Parameter('w_z4'),3)

circuit.draw()

┌─────────┐┌──────────┐┌──────────┐┌──────────┐
q_0: ┤ Ry(i_1) ├┤ Rx(w_x1) ├┤ Ry(w_y1) ├┤ Rz(w_z1) ├
     ├─────────┤├──────────┤├──────────┤├──────────┤
q_1: ┤ Ry(i_2) ├┤ Rx(w_x2) ├┤ Ry(w_y2) ├┤ Rz(w_z2) ├
     ├─────────┤├──────────┤├──────────┤├──────────┤
q_2: ┤ Ry(i_3) ├┤ Rx(w_x3) ├┤ Ry(w_y3) ├┤ Rz(w_z3) ├
     ├─────────┤├──────────┤├──────────┤├──────────┤
q_3: ┤ Ry(i_4) ├┤ Rx(w_x4) ├┤ Ry(w_y4) ├┤ Rz(w_z4) ├
     └─────────┘└──────────┘└──────────┘└──────────┘